# Iterative Prisoner's Dilemma

## Description

## Project's aims

## Project's structure

This project is structured as follow:
* **Point one - Fight 1 vs 1**: development of a method to implement a single IPD problem, between two chosen strategies which "fight" and score points based on the payoff parameters.
* **Point Two - All vs all**: by implementing a round-robin scheme, where each strategy competes against all the others, the general behaviour of each strategy can be studied to observe the best performing one in a single fight.
* **Point Three - Population evolution**: study the efficiency of every strategy in a "long-term" conflict by iterating the fights *all vs all* and evolving the population of the partecipants at each iteration by "killing" the worst performing ones and increasing the numbers of the "winning" ones.
* **Point Four - Mutations**: repetition of the previous process but giving the strategies a probability to mutate at each iteration, with a gene that encodes the  possibility for a partecipant to collaborate independently from his strategy.

### Libraries and imported files

We have implemented some libraries and, in order to simplify and streamline the main project, we have created strategies.py, update_func.py and it_pris_dil_func.py, which we'll discuss later.

In [2]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import it_pris_dil_func as pris_dil

### Strategies implemented

* **Nice guy**: always cooperates
* **Bad guy**: always defects 
* **Mainly nice**: randomly defects $k\%$ of the times and cooperates $100-k\%$, for this project we chose $k = 30$
* **Mainly bad**: randomly defects $k\%$ of the times and cooperates $100-k\%$, for this project we chose $k = 30$
* **Tit-for-tat**: starts by cooperating, then repeats what the opponent has done in the previous move
* **Random**:cooperates or defects randomly
* **Grim**: starts by cooperating, but if the opponent defects one time, it'll always defect
* **Forgiving Tit-Tat**: starts by cooperating, then repeats what the opponent has done in the previous move, but defects only if the opponent defects for two consecutive rounds
* **Suspicious Tit-Tat**: starts by defecting, then repeats what the opponent has done in the previous move
* **Pavlov**: cooperates if in the previous round he and the opponent had done the same move, otherwise defects 
* **Reactive Nice**: in the first round cooperates $50\%$ of the times, then cooperates with probability $p$ if the opponent cooperated in  the previous round, and with probability $q$ if the opponent defected. In this case the probability values $p$ and $q$ were set respectively at $0.7$ and $0.3$
* **Reactive Bad**: same strategy as the previous one, but with set values $p$ and $q$ respectively $0.3$ and $0.7$
* **Hard Joss**: plays like Tit-for-Tat, but cooperates only with probability $0.9$
* **Soft Joss**: plays like Tit-for-Tat, but defects only with probability $0.9$

In [ ]:
uc, ud = [1,0], [0,1]
k=0.3

def nice_guy(it,u,v,u2):
    return uc

def bad_guy(it,u,v,u2):
    return ud

def mainly_nice(it,u,v,u2):
    a = npr.rand()
    if a > k: return uc
    else: return ud
    
def mainly_bad(it,u,v,u2):
    a = npr.rand()
    if a <= k: return uc
    else: return ud
    
def tit_tat(it,u,v,u2):
    if it==2: return uc
    else: return u
    
def random(it,u,v,u2):
    a = npr.rand()
    if a < 0.5: return uc
    else: return ud
    
def pavlov(it,u,v,u2):
    if it==2:
        return uc
    else:
        if v == ud:
            return ud
        else:
            if u == ud:
                return ud
            else: return uc

def f_tit_tat(it,u,v,u2):
    if it <= 3: return uc
    else:
        if u == ud and u2 == ud:
            return ud
        else: return uc

As input variables we have:
* `it`: current iteration (round) of the 1vs1 fight
* `u`: opponent's move in the previous round
* `u2`: opponent's move two rounds before
* `v`: player's move in the previous round

### Strategy dictionary: strat

Spieghiamo come richiamiamo le varie strategie.

In [ ]:
strat = {'nice': partial(st.nice_guy),
        'bad': partial(st.bad_guy), 
        'm_nice': partial(st.mainly_nice),
        'm_bad': partial(st.mainly_bad),
        'tit_tat': partial(st.tit_tat),
        'random': partial(st.random),
        'grim': partial(st.grim),
        'f_tit_tat': partial(st.f_tit_tat),
        'sus_tit_tat': partial(st.sus_tit_tat),
        'pavlov': partial(st.pavlov),
        'reactive_nice': partial(st.reactive_nice),
        'reactive_bad': partial(st.reactive_bad),
        'hard_joss': partial(st.hard_joss),
        'soft_joss': partial(st.soft_joss)}

## Point 1: IPD between two players

In this first part we want to implement a iterated prisoner's dilemma implementing two given strategies.

### 1vs1 Fight function

Here is reported the function to calculate the fight 1vs1 which consists in a game of the standard prisoner dilemma between two given strategies, iterated for a chosen number of ti`mes.

To study the results of a contention between two given strategy we implemented the `fight()` function, which does the following:
* create the **answer vectors** `p1` and `p2`, with the actions of the two strategies at each iteration.
* from them, compute the cumulative vector of **results**

In [ ]:
def fight(player1,player2,N=None):

    if N == None: N = 100
            
    R, S, T, P = 3, 0, 5, 1
    M = np.array([[R,S],[T,P]])

    p1, p2 = [-1,-1], [-1,-1]
    for i in range(2,N+2):
        if probf == 0 and probg == 0:
            p1.append(strat[player1](i,p2[i-1],p1[i-1],p2[i-2]))
            p2.append(strat[player2](i,p1[i-1],p2[i-1],p1[i-2]))

    p1 = np.array(p1[2:]).T
    p2 = np.array(p2[2:]).T

    result_1 = np.cumsum([np.dot(p1[:,i].T,np.dot(M,p2[:,i])) for i in range(N)])
    result_2 = np.cumsum([np.dot(p2[:,i].T,np.dot(M,p1[:,i])) for i in range(N)])

    return result_1[-1], result_2[-1]

For simplicity in this first point we use only the standard strategies, represented in a list called `s`.
We use this function in a nested for loop in order to create the **result matrix**, containing the score of all the fights' combinations.

In [ ]:
s = ['nice','bad','m_nice','m_bad','tit_tat']

result = np.zeros((len(s),len(s)))
for i in range(len(s)):
    for j in range(i,len(s)):
        p1, p2 = pris_dil.fight(s[i],s[j])
        result[i,j] = p1
        result[j,i] = p2

* mostrare come con il programma possiamo creare un df con i risultati degli scontri singoli (img dataframe), prima mostrrare i primi 5, vince bad, poi vedere come con tutte le strategie già le strategie più carine vincono
* esempi di combattimenti 1vs1 con griglia colorata e con grafico generale (tit_tat vs sus_tit_tat, random vs grim, m_bad vs f_tit_tat, pavlov vs soft_joss, reactive_nice vs reactive_nice)
* considerazioni su strategie più efficienti

## Point 2: Multiple Players IPD - MPIPD

Parlare del vettore h, come lo generiamo e come facciamo a collegare un numero intero con una specifica strategia.

### Round Robin function


In [ ]:
def round_robin(h,s, ord=False):

    N = len(h)
    partecipants = [s[int(i)] for i in h] 
    
    h = np.array(h)

    result = np.zeros((N,N))
    somma = np.zeros(N)
    for i in range(N):
        for j in range(i+1,N):
            p1, p2 = fight(partecipants[i],partecipants[j])
            result[i,j] = p1
            result[j,i] = p2

        somma[i] = np.sum(result[i,:])

    unique, n_strategies = np.unique(h,return_counts=True)
    media = np.zeros(len(unique))

    for i in range(N):
        val = int(np.argwhere(unique == h[i]))
        media[val] += somma[i]

    media = np.round(media/n_strategies,2)
    
    if ord == True:
        sort = media.argsort()
        media = media[sort]
        unique = unique[sort]
        n_strategies = n_strategies[sort]

    return unique, media, n_strategies

Spiegazione di r_r

In [ ]:
s = ['nice','bad','m_nice','m_bad','tit_tat']
h = npr.randint(0,len(s),size=30)

unique, media, n_strat = pris_dil.round_robin(h,s,ord=True)

**ESEMPI E CONSIDERAZIONI**
* solo le prime 5 (o assortimento con lo stesso numero di buone e cattive) -> vince bad
* aggingi strategie buone la situa cambia
* migliorare il grafico a barre con l'informazione sul numero di partecipanti

## Point 3: repeated MPIPD - rMPIPD

introdurre tournament

In [ ]:
def tournament(h,update_f,s,it=None,n_change=None):
    
    if it == None: it = 100

    n_matrix = np.zeros([it,len(s)])  #matrix of the number of strategies at each iteration
    val_matrix = np.zeros([it,len(s)])  #matrix of the average scores at each iteration

    for i in range(it):    
        strategies, average_results, numbers = round_robin(h,s,ord=True)

        numbers_1 = np.zeros(len(n_matrix.T))
        average_1 = np.zeros(len(n_matrix.T))
        
        for j in range(len(strategies)):
            numbers_1[strategies[j]] = int(numbers[j])
            average_1[strategies[j]] = average_results[j]

        n_matrix[i] = numbers_1
        val_matrix[i] = average_1

        h = update[update_f](h,strategies,average_results,s,n_change)

    return n_matrix, val_matrix

presentare le varie update a parole (elenco puntato, magari disegnini) e fare considerazioni su quale sia la migliore
* h fisso provare i diversi update, scegliere un h neutro con stesso numero di buoni e cattivi
presentare poi l'update scelto, probably il 5

In [ ]:
s = ['nice','bad','m_nice','m_bad','tit_tat',
    'random','grim','f_tit_tat','sus_tit_tat',
    'pavlov','reactive_nice','reactive_bad',
    'hard_joss','soft_joss']
    
h = npr.randint(0,len(s),size=60)
unique, n_strategies = np.unique(h,return_counts=True)

for a,b in zip(unique,n_strategies): print(s[a],b)

iterations = 100
n_ma3, val_ma3 = pris_dil.tournament(h,'update_3',s,it=iterations)

Scelto update, fare un sacco di test e vedere come si comportano le strategie, trarre le prime conclusioni su chi sia la best della crew
studiare diverse configurazioni di h
* primi 5
* all together now
* distribuzione pensata con meno buoni
* forti vs forti e scarsi vs scarsi
* 70% bad vs 30% tit tat miste
* sus vs forgiving vs tit_tat

## Point 4: rMPIPD with mutations

Presentare l'algoritmo di applicazione del gene, funzione mutate(), spiegare brevemente come abbiamo aggiornato le funzioni precedenti per adattarle a mutate()

In [ ]:
def mutation(player,gene,it,u,v,u2):

    if npr.random() < gene:
        return [1,0]
    else:
        return strat[player](it,u,v,u2)
    
def tournament(h,update_f,s,it=None,mutation_prob=None,n_change=None):
    
    if it == None: it = 100
    s_ref = [[i,0] for i in range(len(s))]

    new_strat = 0
    n_matrix = np.zeros([it,len(s)])                   #matrix of the number of strategies at each iteration
    val_matrix = np.zeros([it,len(s)])                 #matrix of the average scores at each iteration
    new_col = np.zeros((it,1))

    for i in range(it):    
        strategies, average_results, numbers = round_robin(h,s,ord=True)
        
        for j in range(new_strat):                     #adds a new column to n_matrix and val_matrix 
            n_matrix = np.hstack((n_matrix,new_col))   #for each new strategy born in the previous iteration
            val_matrix = np.hstack((val_matrix,new_col))

        numbers_1 = np.zeros(len(n_matrix.T))
        average_1 = np.zeros(len(n_matrix.T))

        if np.shape(h) == (len(h.T),):                 #NO mutations
            for j in range(len(strategies)):
                numbers_1[strategies[j]] = int(numbers[j])
                average_1[strategies[j]] = average_results[j]

        else:
            for j in range(len(strategies.T)):         #YES mutations
                for k in range(len(s_ref)):            #new slicing method adapted for 2-dim h
                    if np.all(s_ref[k] == strategies[:,j]):
                        ind = k
                numbers_1[ind] = int(numbers[j])
                average_1[ind] = average_results[j]

        n_matrix[i] = numbers_1
        val_matrix[i] = average_1

        h, new_strat = update[update_f](h,strategies,average_results,s,s_ref,mutation_prob,n_change)

    return n_matrix, val_matrix

Presentare il nuovo h, s_ref, new_strat e robe varie

In [ ]:
s = ['nice','bad','m_nice','m_bad','tit_tat',
    'random','grim','f_tit_tat','sus_tit_tat',
    'pavlov','reactive_nice','reactive_bad',
    'hard_joss','soft_joss']

n_players = 60
h = np.zeros((2,n_players))
h[0] = npr.randint(0,len(s),size=n_players)
unique, n_strategies = np.unique(h[0],return_counts=True)

for a,b in zip(unique,n_strategies): print(s[int(a)],b)

iterations = 100
n_ma4, val_ma4 = pris_dil.tournament(h,'update_3',s,it=iterations,mutation_prob=0.05)

Poi fare delle nuove prove e simulazioni e trarre le dovute conclusioni
* fissare un h con non troppe strategie diverse e cambiare la prob di mutazione
* prendere ispirazione dalle h del punto prima e studiare casi interessanti
meglio usare h senza troppi partecipanti diversi e con lo stesso numero per non sbilanciare la probabilità di mutazione